#Data Visualization (Final Project-CS 625)
## Headline:
## How Stellar Properties Shape Planetary Systems: From Spectral Types to Planet Temperatures
### By: Bhargav Iyer
### Date: 4/20/2025

### Question 1: How does the number of known planets vary across different spectral types of stars?

In [1]:
spectral_info = {
    'O': "O: Blue, hottest stars (>30,000K), rare and massive",
    'B': "B: Blue-white, very hot (10,000–30,000K)",
    'A': "A: White, strong hydrogen lines (7,500–10,000K)",
    'F': "F: Yellow-white, moderate hydrogen (6,000–7,500K)",
    'G': "G: Yellow, like our Sun (5,200–6,000K)",
    'K': "K: Orange, cooler than Sun (3,700–5,200K)",
    'M': "M: Red, cool stars (2,400–3,700K), very common",
    'L': "L: Infrared, brown dwarfs (1,300–2,400K)",
    'T': "T: Methane dwarfs, very cool (<1,300K)",
    'Y': "Y: Coldest known stars (~<500K)"
}

import plotly.graph_objects as go
import pandas as pd

# Load the dataset
df = pd.read_csv("PSCompPars_2025.04.04_15.46.52.csv", skiprows=88, comment="#")

# Drop rows without key columns
df = df.dropna(subset=['st_spectype', 'hostname', 'st_teff'])

# Group by specific spectral types and host systems
grouped = df.groupby(['st_spectype', 'hostname']).agg({'st_teff': 'mean'}).reset_index()

# Count number of unique systems per spectral type
counts = grouped.groupby('st_spectype').agg({
    'hostname': 'nunique',
    'st_teff': 'mean'
}).reset_index().rename(columns={'hostname': 'num_systems'})

# Extract spectral class (first letter of spectral type)
counts['spectral_class'] = counts['st_spectype'].str[0]

# Define colors for spectral classes (customize as needed)
color_map = {
    'O': '#6A0DAD',
    'B': '#0000FF',
    'A': '#87CEEB',
    'F': '#00FF00',
    'G': '#FFFF00',
    'K': '#FFA500',
    'M': '#FF4500',
    'L': '#FF69B4',
    'T': '#8B008B',
    'Y': '#708090'
}

# Assign colors to each spectral type based on its class
counts['color'] = counts['spectral_class'].map(color_map).fillna('#CCCCCC')  # fallback color

# Get unique spectral classes
spectral_classes = sorted(counts['spectral_class'].unique())

# Prepare the figure
fig = go.Figure()

# Add bar trace for all data (all spectral types together)
fig.add_trace(go.Bar(
    x=counts['st_spectype'],
    y=counts['num_systems'],
    name='All',
    visible=True,
    marker=dict(color=counts['color']),
    hovertext=counts['spectral_class'],
    hoverinfo='x+y+text'
))

# Add one trace per spectral class (initially hidden)
for st_class in spectral_classes:
    filtered = counts[counts['spectral_class'] == st_class]
    fig.add_trace(go.Bar(
        x=filtered['st_spectype'],
        y=filtered['num_systems'],
        name=f"Spectral Class {st_class}",
        visible=False,
        marker=dict(color=filtered['color']),
        hovertext=filtered['spectral_class'],
        hoverinfo='x+y+text'
    ))

# Create dropdown buttons
buttons = []

# Button to show all spectral types
buttons.append(dict(
    label='All',
    method='update',
    args=[{
        'visible': [True] + [False] * len(spectral_classes)
    },
        {'title': 'Number of Planetary Systems by Spectral Type'}]
))

# Buttons for each spectral class
for i, st_class in enumerate(spectral_classes):
    visibility = [False] * (len(spectral_classes) + 1)
    visibility[i + 1] = True  # Only this class's trace visible
    buttons.append(dict(
        label=st_class,
        method='update',
        args=[{'visible': visibility},
              {'title': f'Spectral Types in Class: {st_class}'}]
    ))

# Update layout with dropdown
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=1.15,
        y=1.1,
        showactive=True
    )],
    title="Number of Planetary Systems by Spectral Type",
    xaxis_title="Spectral Type",
    yaxis_title="Number of Planetary Systems"
)

# Build the annotation text from the spectral_info
annotation_text = "<br>".join([v for k, v in spectral_info.items() if k in spectral_classes])

# Add annotation to the layout
fig.update_layout(
    annotations=[
        dict(
            text=f"<b>Spectral Class Key</b><br>{annotation_text}",
            x=1.07, y=1.0, xref="paper", yref="paper",
            showarrow=False,
            align="left",
            bordercolor="black",
            borderwidth=1,
            bgcolor="white",
            font=dict(size=11)
        )
    ]
)

# Show the interactive figure
fig.show()


### Question 2: How does a planet’s equilibrium temperature relate to its host star’s mass and temperature?

In [2]:
import plotly.express as px

# Load the dataset
df = pd.read_csv("PSCompPars_2025.04.04_15.46.52.csv", skiprows = 88, comment="#")

# Clean data for scatter plot
df2 = df.dropna(subset=['pl_eqt', 'st_teff', 'st_mass'])

# Scatter plot
fig2 = px.scatter(df2,
                  x='st_mass',
                  y='pl_eqt',
                  color='st_teff',
                  hover_name='pl_name',
                  labels={'st_mass': 'Star Mass (Solar Masses)',
                          'pl_eqt': 'Planet Equilibrium Temperature (K)',
                          'st_teff': 'Star Temperature (K)'},
                  title='Planet Equilibrium Temperature vs Star Mass')

fig2.update_layout(coloraxis_colorbar=dict(title='Star Temp (K)'),
                   xaxis_title='Star Mass (Solar Masses)',
                   yaxis_title='Planet Equilibrium Temperature (K)')
fig2.show()
